In [53]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os
import pickle
from models.gan.gan import generator, discriminator, StockTimeGan
from utils.log import prepare_logger
from pathlib import Path
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error

plt.style.use("dark_background")
plt.rcParams["figure.figsize"] = (30, 10)
plt.rcParams["figure.dpi"] = 600

logger = prepare_logger("INFO")
# %load_ext blackcellmagic

# %%black for formatting


In [54]:
load_path = Path(os.path.abspath("")).parents[0] / "data" / "scaled_data"
models_path = Path(os.path.abspath("")).parents[0] / "models" / "gan" / "versions"

with open(load_path / "data.pickle", "rb") as handle:
    data = pickle.load(handle)

with open(load_path / "scalers.pickle", "rb") as handle:
    scalers = pickle.load(handle)

with open(models_path / "model_0.6.pickle", "rb") as handle:
    model_data = pickle.load(handle)


In [55]:
test_preds = scalers["y_scaler"].inverse_transform(model_data["test_preds"])
test_true = scalers["y_scaler"].inverse_transform(model_data["actual_values"])


In [56]:
mean_absolute_error(y_true=test_true, y_pred=test_preds)


1.2267532991559318

In [57]:
np.sqrt(mean_squared_error(y_true=test_true, y_pred=test_preds))


1.731920281420292

In [58]:
df = pd.DataFrame(
    {
        "predictions": test_preds.reshape(
            test_preds.shape[0],
        ),
        "actual_values": test_true.reshape(
            test_true.shape[0],
        ),
    }
)


In [59]:
plt.plot(
    test_preds.reshape(
        test_preds.shape[0],
    ),
    color="blue",
    label="Predicted price",
)
plt.plot(
    test_true.reshape(
        test_true.shape[0],
    ),
    color="red",
    label="Actual price",
)
plt.title("EA actual price vs predicted price")
plt.xlabel("Day number")
plt.ylabel("Valuation")
plt.legend()
plt.show()


In [60]:
test_true.mean()


83.6483415539345

In [61]:
np.median(test_true)


83.47000122070312

In [62]:
np.std(test_true)


11.434061630252828

In [63]:
results = {"1000_epochs_no_sentiment": {"mea": 1.2186, "rmse": 1.7288}}


In [64]:
plt.plot(
    model_data['train_history']['gen_loss'],
    color="blue",
    label="Generator loss",
)
plt.plot(
    model_data['train_history']['disc_loss'],
    color="red",
    label="Discriminator Loss",
)
plt.title("Generator and discriminator loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [65]:
model_data['train_history'].keys()

dict_keys(['gen_loss', 'disc_loss', 'real_y', 'pred_y'])

O czym napisac: 
- Spotkanie 
- Problemy ze stabilizacja modelu 
- Wywalenie dense, dodawanie i wywalanie sentymentu 
- Co z innymi modelami 
- Co jeszcze potrzeba 
- Wykresy? 
- Czy inne firmy 